### Prepare Pyspark

**Import Library**

In [1]:
from pyspark import *
from pyspark.sql.functions import *
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.ml.fpm import FPGrowth

**Session Setting**

In [2]:
# Set JAVA_HOME if not already set (update the path to your Java installation if needed)
if "JAVA_HOME" not in os.environ:
    os.environ["JAVA_HOME"] = "C:/Program Files/Java/jdk-17"  # Change this path to your Java installation

# Initialize SparkSession
spark = (
    SparkSession.builder
    .appName("MyPySparkApp")
    .master("local[*]")  # ใช้ทุก core ของเครื่อง
    .getOrCreate()
)

import os

# Set HADOOP_HOME for Windows Spark compatibility
os.environ["HADOOP_HOME"] = "C:/hadoop/bin"  # Change this path to your Hadoop installation

from pyspark.ml.fpm import FPGrowth
from pyspark.sql.functions import col

**Import Data**

In [3]:
#%pip install opendatasets --upgrade
#import opendatasets as od
#od.download("https://www.kaggle.com/datasets/middlehigh/los-angeles-crime-data-from-2000","/Volumes/workspace/default/crime")

**เปิด & อ่านไฟล์**

In [4]:
file_path = "C:/Users/User/Desktop/Association_Rule2/Chicage Crime Data.csv"
df = spark.read.options(header="true", inferschema=True).csv(file_path)

**Preview Data**

In [5]:
df.show()

+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
| 5741943|   HN549294|08/25/2007 09:22:...|  074XX N ROGERS AVE|0560|             ASSAULT|              SIMPLE|             

### Clean Data & Change Data type of Column Date to Date & Timestamp

**นับค่า Null**

In [6]:
import pyspark.sql.functions as F
from functools import reduce  # 👈 ต้องมีบรรทัดนี้นะ

# ฟังก์ชันรวมเงื่อนไขเช็ค null ทุกคอลัมน์
condition = reduce(lambda a, b: a | b, [F.col(c).isNull() for c in df.columns])

# นับจำนวน record ที่มีค่า null อย่างน้อย 1 ช่อง
null_count = df.filter(condition).count()

print(f"มีแถวที่มีค่า NULL อย่างน้อยหนึ่งช่องทั้งหมด {null_count} แถว")


มีแถวที่มีค่า NULL อย่างน้อยหนึ่งช่องทั้งหมด 686725 แถว


**นำค่า Null ออก**

In [7]:
df_clean = df.na.drop()

print("หลังจากลบค่า NULL แล้ว เหลือทั้งหมด:", df_clean.count(), "แถว")

หลังจากลบค่า NULL แล้ว เหลือทั้งหมด: 6704462 แถว


**นับค่าNullหลังUpdateเสร็จแล้ว**

In [8]:
import pyspark.sql.functions as F
from functools import reduce  # 👈 ต้องมีบรรทัดนี้นะ

# ฟังก์ชันรวมเงื่อนไขเช็ค null ทุกคอลัมน์
condition = reduce(lambda a, b: a | b, [F.col(c).isNull() for c in df_clean.columns])

# นับจำนวน record ที่มีค่า null อย่างน้อย 1 ช่อง
null_count = df_clean.filter(condition).count()

print(f"มีแถวที่มีค่า NULL อย่างน้อยหนึ่งช่องทั้งหมด {null_count} แถว")


มีแถวที่มีค่า NULL อย่างน้อยหนึ่งช่องทั้งหมด 0 แถว


**เปลี่ยนFormatวันที่ให้เป็นTimeStamp**

In [9]:
df_with_timestamp = df_clean.withColumn(
    "Timestamp", to_timestamp("Date", "MM/dd/yyyy hh:mm:ss a"))
df_with_timestamp.show()

+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-------------------+
|      ID|Case Number|                Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|          Timestamp|
+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-------------------+
|   25953|   JE240540|05/24/2021 03:06:...| 020XX N LARAMIE AVE|

### Select & filter columns & Clean Data
- Column : ID , Year (2020 - 2024), Timestamp, Primary Type, Community Area, Arrest, Beat
- Year between 2020 - 2024 (5 Years)

**เลือกข้อมูลที่ Arrest = True**

In [16]:
from pyspark.sql.functions import col

# เลือกคอลัมน์และกรองตาม Timestamp + Arrest = true
df_selected = df_with_timestamp.select(
    "ID", "Year", "Timestamp", "Primary Type", "Location Description",
    "Community Area", "Arrest", "Beat"
).filter(
    (col("Timestamp") >= "2013-01-01 00:00:00") &
    (col("Timestamp") <= "2024-12-31 23:59:59") &
    (col("Arrest") == True)
)

# แสดงตัวอย่าง 5 แถว
df_selected.show(5, truncate=False)


+--------+----+-------------------+-----------------------+--------------------+--------------+------+----+
|ID      |Year|Timestamp          |Primary Type           |Location Description|Community Area|Arrest|Beat|
+--------+----+-------------------+-----------------------+--------------------+--------------+------+----+
|25953   |2021|2021-05-24 15:06:00|HOMICIDE               |STREET              |19            |true  |2515|
|26038   |2021|2021-06-26 09:24:00|HOMICIDE               |PARKING LOT         |13            |true  |1711|
|13274752|2023|2023-11-12 07:59:00|BATTERY                |SMALL RETAIL STORE  |44            |true  |632 |
|12218064|2020|2020-10-30 16:30:00|CRIMINAL SEXUAL ASSAULT|RESIDENCE           |45            |true  |411 |
|12342615|2021|2021-04-17 15:20:00|ROBBERY                |RESIDENCE           |44            |true  |631 |
+--------+----+-------------------+-----------------------+--------------------+--------------+------+----+
only showing top 5 rows


**นับจำนวนแถวของข้อมูล**

In [17]:
df_selected.count()

497207

**แสดงตาราง**

In [18]:
df_selected.show()

+--------+----+-------------------+--------------------+--------------------+--------------+------+----+
|      ID|Year|          Timestamp|        Primary Type|Location Description|Community Area|Arrest|Beat|
+--------+----+-------------------+--------------------+--------------------+--------------+------+----+
|   25953|2021|2021-05-24 15:06:00|            HOMICIDE|              STREET|            19|  true|2515|
|   26038|2021|2021-06-26 09:24:00|            HOMICIDE|         PARKING LOT|            13|  true|1711|
|13274752|2023|2023-11-12 07:59:00|             BATTERY|  SMALL RETAIL STORE|            44|  true| 632|
|12218064|2020|2020-10-30 16:30:00|CRIMINAL SEXUAL A...|           RESIDENCE|            45|  true| 411|
|12342615|2021|2021-04-17 15:20:00|             ROBBERY|           RESIDENCE|            44|  true| 631|
|12785595|2022|2022-08-05 21:00:00|         SEX OFFENSE|           APARTMENT|            69|  true| 324|
|12990873|2019|2019-08-17 13:14:00|OFFENSE INVOLVING...

**เลือกข้อมูลมา 5 ปี**

In [19]:
df_FiveYear = df_selected.filter(
    (col('Timestamp') >= '2020-01-01 00:00:00') &
    (col('Timestamp') <= '2024-12-31 23:59:59')
)

**นับจำนวนแถว 5 ปี**

In [20]:
df_FiveYear.count()

49805

**กรองและเตรียมข้อมูล Street ทำเป็นBasketเพื่อนำไปใช้วิเคราะห์ Association**

In [21]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# 1. เลือกเฉพาะข้อมูลที่เกิดใน Apartment
df_street = df_FiveYear.filter(F.col("Location Description") == "STREET")

# 2. สร้างคอลัมน์ 'Year' จากคอลัมน์วันที่ (ถ้ายังไม่มี)
# สมมติคอลัมน์ชื่อ 'Date' หรือ 'Date Occurred' — ปรับตามจริงนะ
df_street = df_street.withColumn("Year", F.year(F.col("Timestamp")))

# 3. กลุ่มข้อมูลตามปี แล้วรวม Primary Type ทั้งหมดในปีนั้นเป็น basket
df_basket_street = (
    df_street.groupBy("Year")
    .agg(F.collect_set("Primary Type").alias("basket"))
    .orderBy("Year")
)

# 4. เพิ่มคอลัมน์ NO (เรียงลำดับปี)
windowSpec = Window.orderBy("Year")
df_basket_street = df_basket_street.withColumn("NO", F.row_number().over(windowSpec))

# 5. เรียงคอลัมน์ให้สวย
df_basket_street = df_basket_street.select("NO", "Year", "basket")

df_basket_street.show(truncate=False)


+---+----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|NO |Year|basket                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
+---+----+------------------------------

### Association Process

In [22]:
from pyspark.sql import functions as F

df_basket_street.select(F.size("basket").alias("len")).agg(F.max("len")).show()
print("จากข้อมูลที่พร้อมทำ Association มีอยู่:", df_basket_street.count(), "แถว")

+--------+
|max(len)|
+--------+
|      28|
+--------+

จากข้อมูลที่พร้อมทำ Association มีอยู่: 5 แถว


**ทำโมเดล Association**

In [23]:
from pyspark.ml.fpm import FPGrowth
from pyspark.sql.functions import col

# --- รัน FP-Growth ---
fpGrowth = FPGrowth(itemsCol="basket", minSupport=0.75, minConfidence=1)
model = fpGrowth.fit(df_basket_street)

**--- นับจำนวน Association Rules ---**

In [24]:

num_rules = model.associationRules.count()
print(f"จำนวน Association Rules: {num_rules}")

จำนวน Association Rules: 10938


### ดูกฎที่ Confidence = 1, Lift > 1, Itemset <= 3 

In [25]:
from pyspark.sql.functions import col, size, concat_ws

# กรองกฎ: confidence = 1, lift > 1, antecedent ≤ 3 items
rules_filtered = model.associationRules.filter(
    (col("confidence") == 1.0) &
    (col("lift") > 1.0) &
    (size(col("antecedent")) <= 3)
)

# รวม antecedent/consequent เป็น string อ่านง่าย
rules_filtered_str = rules_filtered.withColumn("antecedent", concat_ws(", ", col("antecedent"))) \
                                   .withColumn("consequent", concat_ws(", ", col("consequent")))

# สั่ง collect และ sort ตาม lift-desc
all_rules = rules_filtered_str.orderBy(col("lift").desc(), col("confidence").desc()).collect()

# แสดงผล
for row in all_rules:
    print(f"{row['antecedent']} => {row['consequent']} | confidence: {row['confidence']:.2f} | lift: {row['lift']:.2f}")


PUBLIC PEACE VIOLATION, WEAPONS VIOLATION, ROBBERY => NARCOTICS | confidence: 1.00 | lift: 1.25
PUBLIC PEACE VIOLATION, WEAPONS VIOLATION, ROBBERY => DECEPTIVE PRACTICE | confidence: 1.00 | lift: 1.25
PUBLIC PEACE VIOLATION, WEAPONS VIOLATION, ROBBERY => CRIMINAL DAMAGE | confidence: 1.00 | lift: 1.25
PUBLIC PEACE VIOLATION, WEAPONS VIOLATION, ROBBERY => THEFT | confidence: 1.00 | lift: 1.25
PUBLIC PEACE VIOLATION, NARCOTICS, WEAPONS VIOLATION => DECEPTIVE PRACTICE | confidence: 1.00 | lift: 1.25
PUBLIC PEACE VIOLATION, NARCOTICS, WEAPONS VIOLATION => CRIMINAL DAMAGE | confidence: 1.00 | lift: 1.25
PUBLIC PEACE VIOLATION, NARCOTICS, WEAPONS VIOLATION => THEFT | confidence: 1.00 | lift: 1.25
NARCOTICS, HOMICIDE, WEAPONS VIOLATION => DECEPTIVE PRACTICE | confidence: 1.00 | lift: 1.25
NARCOTICS, HOMICIDE, WEAPONS VIOLATION => CRIMINAL DAMAGE | confidence: 1.00 | lift: 1.25
NARCOTICS, HOMICIDE, WEAPONS VIOLATION => THEFT | confidence: 1.00 | lift: 1.25
NARCOTICS, HOMICIDE, WEAPONS VIOLATIO

**Export ข้อมูลเป็น Excel**

In [27]:
# ติดตั้งไลบรารีที่ใช้เขียน Excel (openpyxl) แล้วแปลง all_rules ให้เป็น Pandas DataFrameแล้ว export เป็น Excel ได้
import pandas as pd
rules_list = [(row['antecedent'], row['consequent'], row['confidence'], row['lift']) for row in all_rules]
rules_df = pd.DataFrame(rules_list, columns=['Antecedent', 'Consequent', 'Confidence', 'Lift'])
rules_df.to_excel("association_rules_Street.xlsx", index=False)
